# From artisan

In [2]:
<?php

define('LARAVEL_START', microtime(true));

/*
|--------------------------------------------------------------------------
| Register The Auto Loader
|--------------------------------------------------------------------------
|
| Composer provides a convenient, automatically generated class loader
| for our application. We just need to utilize it! We'll require it
| into the script here so that we do not have to worry about the
| loading of any of our classes manually. It's great to relax.
|
*/

require __DIR__.'/vendor/autoload.php';

$app = require_once __DIR__.'/bootstrap/app.php';

/*
|--------------------------------------------------------------------------
| Run The Artisan Application
|--------------------------------------------------------------------------
|
| When we run the console application, the current CLI command will be
| executed in this console and the response sent back to a terminal
| or another output device for the developers. Here goes nothing!
|
*/

$kernel = $app->make(Illuminate\Contracts\Console\Kernel::class);

$status = $kernel->handle(
    $input = new Symfony\Component\Console\Input\ArgvInput,
    new Symfony\Component\Console\Output\ConsoleOutput
);

<warning>PHP Deprecated:  Using ${var} in strings is deprecated, use {$var} instead in /home/mark/www/housing-prices/vendor/voku/portable-utf8/src/voku/helper/UTF8.php on line 3880</warning>
<warning>PHP Deprecated:  Using ${var} in strings is deprecated, use {$var} instead in /home/mark/www/housing-prices/vendor/voku/portable-utf8/src/voku/helper/UTF8.php on line 4846</warning>
<warning>PHP Deprecated:  Using ${var} in strings is deprecated, use {$var} instead in /home/mark/www/housing-prices/vendor/voku/portable-utf8/src/voku/helper/UTF8.php on line 11473</warning>
<warning>PHP Deprecated:  Using ${var} in strings is deprecated, use {$var} instead in /home/mark/www/housing-prices/vendor/voku/portable-utf8/src/voku/helper/UTF8.php on line 11473</warning>
1


# Import

In [28]:
use Illuminate\Console\Command;
use Illuminate\Support\Carbon;
use Illuminate\Support\Facades\DB;
use Rubix\ML\CrossValidation\Metrics\SMAPE;
use Rubix\ML\CrossValidation\Reports\ErrorAnalysis;
use Rubix\ML\Datasets\Labeled;
use Rubix\ML\Datasets\Unlabeled;
use Rubix\ML\GridSearch;
use Rubix\ML\PersistentModel;
use Rubix\ML\Persisters\Filesystem;
use Rubix\ML\Regressors\GradientBoost;
use Rubix\ML\Regressors\RegressionTree;

# Seed the random number generator

In [4]:
srand(100);

NULL


# Fetch data from the house_sales table

In [5]:
$data = DB::table('house_sales')->select(
    'date',
    'bedrooms',
    'bathrooms',
    'sqft_living',
    'sqft_lot',
    'floors',
    'waterfront',
    'view',
    'condition',
    'grade',
    'sqft_above',
    'sqft_basement',
    'yr_built',
    'yr_renovated',
    'zipcode',
    'price'
)->get();

echo "Loaded house sale data";

Loaded house sale data


# Get average values

In [17]:
$data->avg('sqft_living');

2079.8997362698374


# Build dataset

In [19]:
$samples = $data->map(function($sample) {
    return [
        'date' => Carbon::parse($sample->date)->timestamp,
        'bedrooms' => (float) $sample->bedrooms,
        'bathrooms' => (float) $sample->bathrooms,
        'sqft_living' => $sample->sqft_living,
        'sqft_lot' => $sample->sqft_lot,
        'floors' => (float) $sample->floors,
        'waterfront' => $sample->waterfront,
        'view' => (string) $sample->view,
        'condition' => $sample->condition,
        'grade' => $sample->grade,
        'sqft_above' => $sample->sqft_above,
        'sqft_basement' => $sample->sqft_basement,
        'yr_built' => (float) $sample->yr_built,
        'yr_renovated' => (float) $sample->yr_renovated,
        'zipcode' => $sample->zipcode,
    ];
})->toArray();

$labels = $data->pluck('price')->toArray();

$dataset = Labeled::build($samples, $labels);

echo "Dataset built";

Dataset built


# Build training/testing data split

In [20]:
[$training, $testing] = $dataset->randomize()->split(0.8);
echo "Training and testing data split built";

Training and testing data split built


# Train RegressionTree

In [22]:
$estimator = new RegressionTree();

$estimator->train($training);

echo "Estimator trained"

Estimator trained


# Make predictions on testing data and get error report

In [26]:
$predictions = $estimator->predict($testing);

$report = new ErrorAnalysis();

$report->generate($predictions, $testing->labels())

\Rubix\ML\Report::__set_state(array(
   'attributes' => 
  array (
    'mean absolute error' => 145358.1455095468,
    'median absolute error' => 92000.0,
    'mean squared error' => 57433076827.45519,
    'mean squared log error' => 0.13142180528394265,
    'mean absolute percentage error' => 28.57647834340099,
    'rms error' => 239651.991077594,
    'r squared' => 0.5276647035412245,
    'error mean' => 2963.6220918383096,
    'error standard deviation' => 239633.66577246928,
    'error skewness' => 0.8134005564414655,
    'error kurtosis' => 17.948701912909677,
    'error min' => -1910166.6666666665,
    'error 25%' => -93575.0,
    'error median' => -1773.5,
    'error 75%' => 90698.00000000001,
    'error max' => 2789000.0,
    'cardinality' => 4323,
  ),
))


# Make prediction on sample from the database

In [46]:
$dbtest = DB::table('house_sales')->find('01GYV0HZTVFE1YEAMK39NN8DZD');

(object) array(
   'id' => '01GYV0HZTVFE1YEAMK39NN8DZD',
   'house_id' => '7237550310',
   'date' => '2014-05-12',
   'price' => 1225000,
   'bedrooms' => '4.00',
   'bathrooms' => '4.50',
   'sqft_living' => 5420,
   'sqft_lot' => 101930,
   'floors' => '1.00',
   'waterfront' => '0',
   'view' => 0,
   'condition' => '3',
   'grade' => '11',
   'sqft_above' => 3890,
   'sqft_basement' => 1530,
   'yr_built' => 2001,
   'yr_renovated' => 0,
   'zipcode' => '98053',
   'lat' => '47.6561',
   'long' => '-122.005',
   'sqft_living15' => 4760,
   'sqft_lot15' => 101930,
   'created_at' => '2023-04-25 01:30:55',
   'updated_at' => '2023-05-17 02:11:07',
   'estimated_price' => 1215171,
)


In [47]:
$dbsample = [[
    'date' => Carbon::parse($dbtest->date)->timestamp,
    'bedrooms' => (float) $dbtest->bedrooms,
    'bathrooms' => (float) $dbtest->bathrooms,
    'sqft_living' => $dbtest->sqft_living,
    'sqft_lot' => $dbtest->sqft_lot,
    'floors' => (float) $dbtest->floors,
    'waterfront' => $dbtest->waterfront,
    'view' => (string) $dbtest->view,
    'condition' => $dbtest->condition,
    'grade' => $dbtest->grade,
    'sqft_above' => $dbtest->sqft_above,
    'sqft_basement' => $dbtest->sqft_basement,
    'yr_built' => (float) $dbtest->yr_built,
    'yr_renovated' => (float) $dbtest->yr_renovated,
    'zipcode' => $dbtest->zipcode,
]];

$dbtestDataset = Unlabeled::build($dbsample);

echo "Dataset built";

Dataset built


In [48]:
$estimator->predict($dbtestDataset);

array (
  0 => 1225000.0,
)
